# Chapter 4: Building Your First Pose Graph Optimizer

## 🎯 학습 목표

이 챕터에서는 첫 번째 Pose Graph Optimizer를 처음부터 구현합니다. nano-pgo의 핵심 구조를 이해하고 직접 만들어봅니다.

- Pose Graph Optimization의 수학적 기초
- H 행렬과 b 벡터 구성
- Sparse linear system 구축
- Gauss-Newton 최적화 알고리즘
- 실제 데이터로 테스트

## 📚 이론적 배경

### Pose Graph Optimization 문제

최소화할 목적 함수:
$$F(x) = \sum_{ij} e_{ij}^T \Omega_{ij} e_{ij}$$

여기서:
- $e_{ij}$: 포즈 $i$와 $j$ 사이의 에러
- $\Omega_{ij}$: Information matrix (가중치)
- $x$: 모든 포즈의 상태 벡터

### Gauss-Newton 알고리즘

반복적으로 다음을 해결:
$$H \Delta x = -b$$

여기서:
- $H = \sum_{ij} J_{ij}^T \Omega_{ij} J_{ij}$ (Hessian)
- $b = \sum_{ij} J_{ij}^T \Omega_{ij} e_{ij}$ (gradient)
- $J_{ij}$: 에러의 Jacobian

## 🔧 필요한 라이브러리 임포트

In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
from pathlib import Path
import time

# 이전 챕터의 유틸리티 함수들
def rotvec_to_rotmat(rotvec):
    return Rotation.from_rotvec(rotvec).as_matrix()

def rotmat_to_rotvec(R):
    return Rotation.from_matrix(R).as_rotvec()

def skew_symmetric(v):
    """3D 벡터를 skew-symmetric 행렬로 변환"""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

print("✅ 라이브러리 준비 완료!")

## 1. 기본 Pose Graph Optimizer 구조

먼저 optimizer의 기본 구조를 만들어봅시다.

In [ ]:
class SimplePoseGraphOptimizer:
    """간단한 Pose Graph Optimizer
    
    이 클래스는 교육 목적으로 핵심 기능만 구현합니다.
    """
    
    def __init__(self, max_iterations=50, convergence_threshold=1e-4):
        self.max_iterations = max_iterations
        self.convergence_threshold = convergence_threshold
        
        # 상태 변수
        self.poses = {}  # {id: {'t': translation, 'r': rotation_vector}}
        self.edges = []  # [{'from', 'to', 't', 'r', 'information'}]
        self.fixed_poses = set()  # 고정된 포즈 ID들
        
        # 최적화 통계
        self.iteration_stats = []
        
    def add_pose(self, pose_id, translation, rotation_vector):
        """포즈 추가"""
        self.poses[pose_id] = {
            't': np.array(translation),
            'r': np.array(rotation_vector)
        }
        
    def add_edge(self, from_id, to_id, translation, rotation_vector, information=None):
        """엣지(제약) 추가"""
        if information is None:
            information = np.eye(6)  # 기본 information matrix
            
        self.edges.append({
            'from': from_id,
            'to': to_id,
            't': np.array(translation),
            'r': np.array(rotation_vector),
            'information': information
        })
        
    def fix_pose(self, pose_id):
        """포즈를 고정 (gauge freedom 해결)"""
        self.fixed_poses.add(pose_id)
        
    def get_state_vector(self):
        """모든 포즈를 하나의 상태 벡터로 변환"""
        pose_ids = sorted(self.poses.keys())
        state = []
        
        for pid in pose_ids:
            pose = self.poses[pid]
            state.extend(pose['t'])  # translation (3)
            state.extend(pose['r'])  # rotation (3)
            
        return np.array(state), pose_ids
        
    def update_poses_from_state(self, state, pose_ids):
        """상태 벡터로부터 포즈 업데이트"""
        for i, pid in enumerate(pose_ids):
            start_idx = i * 6
            self.poses[pid]['t'] = state[start_idx:start_idx+3]
            self.poses[pid]['r'] = state[start_idx+3:start_idx+6]

print("✅ SimplePoseGraphOptimizer 클래스 정의 완료!")

## 2. Residual과 Jacobian 계산

Between factor의 residual과 Jacobian을 계산하는 함수를 구현합니다.

In [ ]:
def compute_relative_pose_error(pose_i, pose_j, edge):
    """상대 포즈 에러 계산
    
    Args:
        pose_i, pose_j: 포즈 딕셔너리 {'t': translation, 'r': rotation_vector}
        edge: 엣지 딕셔너리 {'t': translation, 'r': rotation_vector}
        
    Returns:
        residual: 6D 에러 벡터 [translation_error, rotation_error]
        Ji, Jj: 포즈 i, j에 대한 Jacobian (6x6)
    """
    # 회전 행렬로 변환
    Ri = rotvec_to_rotmat(pose_i['r'])
    Rj = rotvec_to_rotmat(pose_j['r'])
    Rij_meas = rotvec_to_rotmat(edge['r'])
    
    ti = pose_i['t']
    tj = pose_j['t']
    tij_meas = edge['t']
    
    # 예측된 상대 변환
    Rij_pred = Ri.T @ Rj
    tij_pred = Ri.T @ (tj - ti)
    
    # 에러 계산
    R_error = Rij_meas.T @ Rij_pred
    t_error = Rij_meas.T @ (tij_pred - tij_meas)
    
    # 회전 에러를 rotation vector로
    r_error = rotmat_to_rotvec(R_error)
    
    # Residual: [translation_error, rotation_error]
    residual = np.concatenate([t_error, r_error])
    
    # Jacobian 계산 (근사)
    # ∂e/∂xi
    Ji = np.zeros((6, 6))
    Ji[:3, :3] = -Rij_meas.T @ Ri.T  # ∂t_error/∂ti
    Ji[:3, 3:] = Rij_meas.T @ Ri.T @ skew_symmetric(tj - ti)  # ∂t_error/∂ri
    Ji[3:, 3:] = -np.eye(3)  # ∂r_error/∂ri (근사)
    
    # ∂e/∂xj  
    Jj = np.zeros((6, 6))
    Jj[:3, :3] = Rij_meas.T @ Ri.T  # ∂t_error/∂tj
    Jj[3:, 3:] = np.eye(3)  # ∂r_error/∂rj (근사)
    
    return residual, Ji, Jj

# 테스트
pose_i_test = {'t': np.array([0, 0, 0]), 'r': np.array([0, 0, 0])}
pose_j_test = {'t': np.array([1, 0, 0]), 'r': np.array([0, 0, 0.1])}
edge_test = {'t': np.array([1, 0, 0]), 'r': np.array([0, 0, 0])}

residual, Ji, Jj = compute_relative_pose_error(pose_i_test, pose_j_test, edge_test)
print("📐 Residual과 Jacobian 계산 테스트:")
print(f"   Residual: {residual}")
print(f"   Ji shape: {Ji.shape}")
print(f"   Jj shape: {Jj.shape}")

## 3. H 행렬과 b 벡터 구축

이제 전체 시스템의 H 행렬과 b 벡터를 구축합니다.

In [ ]:
def build_linear_system(poses, edges, fixed_poses):
    """선형 시스템 H*dx = -b 구축
    
    Returns:
        H: Hessian matrix (sparse)
        b: gradient vector
        total_error: 현재 총 에러
        pose_ids: 포즈 ID 순서
    """
    # 포즈 순서 정하기
    pose_ids = sorted(poses.keys())
    pose_idx_map = {pid: idx for idx, pid in enumerate(pose_ids)}
    n_poses = len(poses)
    dim = 6  # 포즈당 차원 (3 translation + 3 rotation)
    
    # Sparse matrix를 위한 리스트
    H_row = []
    H_col = []
    H_data = []
    b = np.zeros(n_poses * dim)
    total_error = 0.0
    
    # 각 엣지에 대해
    for edge in edges:
        from_id = edge['from']
        to_id = edge['to']
        
        if from_id not in poses or to_id not in poses:
            continue
            
        # 포즈 인덱스
        i = pose_idx_map[from_id]
        j = pose_idx_map[to_id]
        
        # Residual과 Jacobian 계산
        residual, Ji, Jj = compute_relative_pose_error(
            poses[from_id], poses[to_id], edge
        )
        
        # Information matrix
        omega = edge['information']
        
        # 에러 누적
        total_error += residual.T @ omega @ residual
        
        # H와 b에 기여분 추가
        # H_ii += Ji^T * Omega * Ji
        Hii = Ji.T @ omega @ Ji
        for row in range(dim):
            for col in range(dim):
                H_row.append(i*dim + row)
                H_col.append(i*dim + col)
                H_data.append(Hii[row, col])
        
        # H_jj += Jj^T * Omega * Jj
        Hjj = Jj.T @ omega @ Jj
        for row in range(dim):
            for col in range(dim):
                H_row.append(j*dim + row)
                H_col.append(j*dim + col)
                H_data.append(Hjj[row, col])
        
        # H_ij += Ji^T * Omega * Jj
        Hij = Ji.T @ omega @ Jj
        for row in range(dim):
            for col in range(dim):
                H_row.append(i*dim + row)
                H_col.append(j*dim + col)
                H_data.append(Hij[row, col])
                # H는 대칭이므로
                H_row.append(j*dim + col)
                H_col.append(i*dim + row)
                H_data.append(Hij[row, col])
        
        # b_i += Ji^T * Omega * residual
        bi = Ji.T @ omega @ residual
        b[i*dim:(i+1)*dim] += bi
        
        # b_j += Jj^T * Omega * residual
        bj = Jj.T @ omega @ residual
        b[j*dim:(j+1)*dim] += bj
    
    # 고정된 포즈 처리 (큰 대각 값 추가)
    fixed_weight = 1e10
    for pose_id in fixed_poses:
        if pose_id in pose_idx_map:
            idx = pose_idx_map[pose_id]
            for k in range(dim):
                H_row.append(idx*dim + k)
                H_col.append(idx*dim + k)
                H_data.append(fixed_weight)
    
    # Sparse matrix 생성
    H = sp.coo_matrix((H_data, (H_row, H_col)), 
                      shape=(n_poses*dim, n_poses*dim))
    H = H.tocsr()  # CSR 포맷으로 변환 (solver용)
    
    return H, b, total_error, pose_ids

print("✅ 선형 시스템 구축 함수 완료!")

## 4. 최적화 알고리즘 구현

Gauss-Newton 알고리즘을 구현합니다.

In [ ]:
class SimplePoseGraphOptimizer(SimplePoseGraphOptimizer):
    """최적화 메서드 추가"""
    
    def optimize(self):
        """Gauss-Newton 최적화 실행"""
        print("🚀 최적화 시작...\n")
        
        # 초기 상태
        state, pose_ids = self.get_state_vector()
        
        for iteration in range(self.max_iterations):
            # H와 b 구축
            H, b, error, _ = build_linear_system(
                self.poses, self.edges, self.fixed_poses
            )
            
            # 통계 저장
            self.iteration_stats.append({
                'iteration': iteration,
                'error': error,
                'gradient_norm': np.linalg.norm(b)
            })
            
            print(f"Iteration {iteration:3d}: error = {error:.6f}, |g| = {np.linalg.norm(b):.6f}")
            
            # 수렴 체크
            if np.linalg.norm(b) < self.convergence_threshold:
                print("\n✅ 수렴 완료!")
                break
            
            # 선형 시스템 해결: H * dx = -b
            try:
                dx = spla.spsolve(H, -b)
            except:
                print("❌ 선형 시스템 해결 실패")
                break
            
            # 상태 업데이트
            state += dx
            self.update_poses_from_state(state, pose_ids)
            
            # dx가 너무 작으면 종료
            if np.linalg.norm(dx) < 1e-6:
                print("\n✅ 변화량이 충분히 작음 - 종료")
                break
        
        return self.iteration_stats
    
    def plot_optimization_progress(self):
        """최적화 진행 상황 시각화"""
        if not self.iteration_stats:
            print("최적화를 먼저 실행하세요!")
            return
            
        stats = self.iteration_stats
        iterations = [s['iteration'] for s in stats]
        errors = [s['error'] for s in stats]
        gradients = [s['gradient_norm'] for s in stats]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Error plot
        ax1.plot(iterations, errors, 'b-o')
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Total Error')
        ax1.set_title('Optimization Progress - Error')
        ax1.grid(True, alpha=0.3)
        ax1.set_yscale('log')
        
        # Gradient norm plot
        ax2.plot(iterations, gradients, 'r-o')
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Gradient Norm')
        ax2.set_title('Optimization Progress - Gradient')
        ax2.grid(True, alpha=0.3)
        ax2.set_yscale('log')
        
        plt.tight_layout()
        plt.show()

print("✅ 최적화 알고리즘 구현 완료!")

## 5. 테스트: 간단한 예제

작은 예제로 optimizer를 테스트해봅시다.

In [ ]:
# Optimizer 생성
optimizer = SimplePoseGraphOptimizer(max_iterations=20)

# 사각형 경로 생성 (노이즈 포함)
np.random.seed(42)
noise_trans = 0.1
noise_rot = 0.05

# 포즈 추가 (초기 추정값에 노이즈)
optimizer.add_pose(0, [0 + np.random.randn()*noise_trans, 
                      0 + np.random.randn()*noise_trans, 0], 
                   [0, 0, 0])
optimizer.add_pose(1, [1 + np.random.randn()*noise_trans, 
                      0 + np.random.randn()*noise_trans, 0], 
                   [0, 0, np.random.randn()*noise_rot])
optimizer.add_pose(2, [1 + np.random.randn()*noise_trans, 
                      1 + np.random.randn()*noise_trans, 0], 
                   [0, 0, np.pi/2 + np.random.randn()*noise_rot])
optimizer.add_pose(3, [0 + np.random.randn()*noise_trans, 
                      1 + np.random.randn()*noise_trans, 0], 
                   [0, 0, np.pi + np.random.randn()*noise_rot])

# 엣지 추가 (측정값)
# Odometry edges (높은 신뢰도)
odom_info = np.diag([100, 100, 100, 100, 100, 100])
optimizer.add_edge(0, 1, [1, 0, 0], [0, 0, 0], odom_info)
optimizer.add_edge(1, 2, [0, 1, 0], [0, 0, np.pi/2], odom_info)
optimizer.add_edge(2, 3, [-1, 0, 0], [0, 0, np.pi/2], odom_info)
optimizer.add_edge(3, 0, [0, -1, 0], [0, 0, np.pi/2], odom_info)

# Loop closure edge (약간 낮은 신뢰도)
loop_info = np.diag([50, 50, 50, 50, 50, 50])
optimizer.add_edge(0, 2, [1, 1, 0], [0, 0, np.pi/2], loop_info)

# 첫 번째 포즈 고정
optimizer.fix_pose(0)

# 최적화 전 시각화
def plot_poses(poses, edges, title):
    plt.figure(figsize=(8, 8))
    
    # 포즈 그리기
    for pid, pose in poses.items():
        x, y = pose['t'][:2]
        theta = pose['r'][2]  # z축 회전
        
        # 포즈 위치
        plt.plot(x, y, 'ro', markersize=10)
        plt.text(x+0.05, y+0.05, f'{pid}', fontsize=12)
        
        # 방향 표시
        dx = 0.2 * np.cos(theta)
        dy = 0.2 * np.sin(theta)
        plt.arrow(x, y, dx, dy, head_width=0.05, head_length=0.05, fc='b', ec='b')
    
    # 엣지 그리기
    for edge in edges:
        if edge['from'] in poses and edge['to'] in poses:
            p1 = poses[edge['from']]['t'][:2]
            p2 = poses[edge['to']]['t'][:2]
            
            if abs(edge['from'] - edge['to']) == 1 or \
               (edge['from'] == 0 and edge['to'] == 3) or \
               (edge['from'] == 3 and edge['to'] == 0):
                plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'g-', linewidth=2, alpha=0.7)
            else:
                plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'b--', linewidth=2, alpha=0.7)
    
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.title(title)
    plt.show()

plot_poses(optimizer.poses, optimizer.edges, 'Before Optimization')

In [ ]:
# 최적화 실행
stats = optimizer.optimize()

# 최적화 후 시각화
plot_poses(optimizer.poses, optimizer.edges, 'After Optimization')

# 최적화 진행 상황
optimizer.plot_optimization_progress()

## 6. 실제 데이터셋으로 테스트

이제 실제 g2o 파일을 로드하여 최적화해봅시다.

In [ ]:
# Chapter 2에서 만든 g2o 파서 재사용
def parse_g2o_simple(filename):
    """간단한 g2o 파서"""
    poses = {}
    edges = []
    
    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue
                
            if tokens[0] == 'VERTEX_SE2':
                pose_id = int(tokens[1])
                x, y, theta = map(float, tokens[2:5])
                poses[pose_id] = {
                    't': np.array([x, y, 0]),
                    'r': np.array([0, 0, theta])
                }
                
            elif tokens[0] == 'EDGE_SE2':
                from_id = int(tokens[1])
                to_id = int(tokens[2])
                dx, dy, dtheta = map(float, tokens[3:6])
                
                # Information matrix (상삼각 부분)
                if len(tokens) >= 12:
                    info_data = list(map(float, tokens[6:12]))
                    # 3x3 SE(2) information을 6x6 SE(3)로 확장
                    info = np.eye(6)
                    info[0, 0] = info_data[0]  # x
                    info[1, 1] = info_data[2]  # y  
                    info[5, 5] = info_data[5]  # theta
                else:
                    info = np.eye(6) * 100
                    
                edges.append({
                    'from': from_id,
                    'to': to_id,
                    't': np.array([dx, dy, 0]),
                    'r': np.array([0, 0, dtheta]),
                    'information': info
                })
                
    return poses, edges

# 파일이 있다면 로드
data_file = Path("../data/input_INTEL_g2o.g2o")
if data_file.exists():
    print("📂 INTEL 데이터셋 로드 중...")
    poses_intel, edges_intel = parse_g2o_simple(str(data_file))
    
    # 큰 데이터셋이므로 일부만 사용
    max_pose_id = 50
    poses_subset = {k: v for k, v in poses_intel.items() if k < max_pose_id}
    edges_subset = [e for e in edges_intel if e['from'] < max_pose_id and e['to'] < max_pose_id]
    
    print(f"✅ 로드 완료: {len(poses_subset)} poses, {len(edges_subset)} edges")
    
    # Optimizer 생성 및 설정
    intel_optimizer = SimplePoseGraphOptimizer(max_iterations=30)
    
    # 포즈 추가
    for pid, pose in poses_subset.items():
        intel_optimizer.add_pose(pid, pose['t'], pose['r'])
    
    # 엣지 추가
    for edge in edges_subset:
        intel_optimizer.add_edge(
            edge['from'], edge['to'],
            edge['t'], edge['r'],
            edge['information']
        )
    
    # 첫 번째 포즈 고정
    intel_optimizer.fix_pose(0)
    
    # 최적화 전 시각화
    plot_poses(intel_optimizer.poses, intel_optimizer.edges, 
               'INTEL Dataset - Before Optimization (First 50 poses)')
    
    # 최적화 실행
    print("\n🚀 INTEL 데이터셋 최적화 시작...\n")
    intel_stats = intel_optimizer.optimize()
    
    # 최적화 후 시각화
    plot_poses(intel_optimizer.poses, intel_optimizer.edges,
               'INTEL Dataset - After Optimization (First 50 poses)')
    
    # 진행 상황 플롯
    intel_optimizer.plot_optimization_progress()
    
else:
    print("⚠️  INTEL 데이터셋을 찾을 수 없습니다.")
    print("   nano-pgo/data 디렉토리를 확인하세요.")

## 7. Sparse Matrix 분석

H 행렬의 sparsity pattern을 확인해봅시다.

In [ ]:
def visualize_hessian_sparsity(optimizer):
    """Hessian 행렬의 sparsity pattern 시각화"""
    H, _, _, _ = build_linear_system(
        optimizer.poses, optimizer.edges, optimizer.fixed_poses
    )
    
    plt.figure(figsize=(10, 10))
    plt.spy(H, markersize=1)
    plt.title(f'Hessian Sparsity Pattern\n{H.shape[0]}×{H.shape[1]} matrix, {H.nnz} non-zeros ({H.nnz/H.shape[0]**2*100:.2f}%)')
    plt.xlabel('Variable Index')
    plt.ylabel('Variable Index')
    
    # 포즈 경계 표시
    n_poses = len(optimizer.poses)
    for i in range(1, n_poses):
        plt.axhline(y=i*6, color='r', linewidth=0.5, alpha=0.5)
        plt.axvline(x=i*6, color='r', linewidth=0.5, alpha=0.5)
    
    plt.show()
    
    # 통계
    print(f"\n📊 Hessian 행렬 통계:")
    print(f"   크기: {H.shape}")
    print(f"   Non-zero 원소: {H.nnz}")
    print(f"   Sparsity: {(1 - H.nnz/H.shape[0]**2)*100:.2f}%")
    print(f"   평균 행당 non-zeros: {H.nnz/H.shape[0]:.1f}")

# 작은 예제의 Hessian
print("🔍 작은 예제 (사각형 경로):")
visualize_hessian_sparsity(optimizer)

# INTEL 데이터셋의 Hessian (로드했다면)
if 'intel_optimizer' in locals():
    print("\n🔍 INTEL 데이터셋:")
    visualize_hessian_sparsity(intel_optimizer)

## 8. 요약 및 핵심 포인트

### 🎓 이 챕터에서 배운 내용:

1. **Pose Graph Optimization의 구조**
   - 상태 벡터: 모든 포즈의 위치와 방향
   - 제약 조건: 포즈 간의 상대 측정값
   - 목적 함수: 가중 최소 제곱 문제

2. **선형 시스템 구축**
   - Residual 계산: 예측값과 측정값의 차이
   - Jacobian 계산: 에러의 미분
   - H와 b 조립: 모든 제약의 기여분 합산

3. **Sparse 행렬 활용**
   - Pose graph의 H는 매우 sparse함
   - Sparse solver로 효율적 해결
   - 메모리와 계산 시간 절약

4. **Gauss-Newton 알고리즘**
   - 반복적 선형화와 업데이트
   - 수렴 조건과 종료 기준
   - 고정 포즈로 gauge freedom 해결

### 💡 다음 챕터 예고:

다음 챕터에서는 Jacobian 계산의 정확도를 높이고, 수동 계산과 Symforce의 자동 계산을 비교해봅니다.

## 🏋️ 연습 문제

### 문제 1: 3D Pose Graph
현재 구현은 주로 2D(SE2)에 초점을 맞추고 있습니다. 완전한 3D(SE3) 지원을 추가해보세요.

### 문제 2: Levenberg-Marquardt
현재는 Gauss-Newton을 사용합니다. Levenberg-Marquardt damping을 추가해보세요.

### 문제 3: 다양한 Factor 타입
Between factor 외에 prior factor, GPS factor 등을 추가해보세요.

In [ ]:
# 여기에 연습 문제를 풀어보세요!

# 문제 2 예시: Levenberg-Marquardt
class LMPoseGraphOptimizer(SimplePoseGraphOptimizer):
    def __init__(self, max_iterations=50, convergence_threshold=1e-4, lambda_init=0.01):
        super().__init__(max_iterations, convergence_threshold)
        self.lambda_lm = lambda_init  # LM damping parameter
        
    def optimize(self):
        """Levenberg-Marquardt 최적화"""
        # 여기에 구현
        pass
